In [ ]:
from datasets import load_dataset
import pandas as pd
import re

#new_folio = load_dataset('fol-autoformalization/FOLIO', token = hf_token)
#new_folio

tran_ds = pd.read_csv(r'/home/flopezp/Kurosagol/Ongoing/datasets/gpt_qwen_translations.csv')
tran_ds = tran_ds.drop(columns=["Unnamed: 0"])
tran_ds.head()

,story_id,premises,premises-FOL,conclusion,conclusion-FOL,label,example_id,gpt Translations,qwen Translations
0,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete.,Athlete(stephenCurry),Uncertain,828,∀x (ProSoccerPlayer(x) → ¬PlayBasketball(x))\n...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
1,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,Stephen Curry is an athlete and a professional...,Athlete(stephenCurry) ∧ Professional(stephenCu...,False,829,∀x (SoccerPlayer(x) → ¬BasketballPlayer(x)) \...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬PlaysBa...
2,324,No professional soccer players play profession...,∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Pla...,"If Stephen Curry is a professional centerback,...",(Professional(stephenCurry) ∧ CenterBack(steph...,True,830,FOL Premises:\n∃x (Athlete(x) ∧ CenterBack(x))...,\n\n∀x (ProfessionalSoccerPlayer(x) → ¬Profess...
3,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",Tyler joined the student government in high sc...,"JoinIn(tyler, studentGovernment, highSchool)",Uncertain,947,∀x (SG(x) → Opportunity(x)) \n∀x (Alone(x) → ...,\n\n∀x (JoinStudentGovernment(x) → Opportuniti...
4,357,If people join student government in high scho...,"∀x (Join(x, studentGovernment) → ManyOpportuni...",If Tyler either both stays alone in high schoo...,"¬(StayAloneIn(tyler, highSchool) ⊕ (Well-adjus...",True,948,∀x (JoinSG(x) → ManyOpportunities(x)) \n∀x (S...,\n\n∀x (JoinSG(x) → Opportunities(x)) \n∀x (S...


In [2]:
def get_nice_dict(regex, instance, const):
	"""
		A partir de una regex, extrae todos los valores encontrados y los formatea para tener un diccionario con apariciones.

		regex = str ; Expresión regular a buscar.
		const = bool ; Determina si estamos analizando constantes. Esto permite que se realice un filtro extra en el código.
	"""
	lista = []
	for _ in instance:
		aux = re.finditer(regex, _)
		for expression in aux:
			if const:
				regex_list = expression.group()[1:-1]
				regex_list = regex_list.split(',')
				for j in regex_list:
					if len(j) > 1:
						constant = re.sub(' ', '', j)
						lista.append(constant)
			else:
				lista.append(expression.group())

	set_set = list(set(lista))
	dict_dict = {}
	for _ in set_set:
		dict_dict["{}".format(_)] = lista.count(_)

	return dict_dict


def extract_info(ds_answer, llm):
	"""
		Limpia la respuesta de un dataset y extrae las premisas necesarias para calcular LogicSim.

		llm = Bool ; Señala si se va a evaluar la respuesta generada por un LLM.
	"""
	instance = ds_answer.split('\n')
	
    # Este bloque tal vez no es necesario para las respuestas de los modelos nuevos.
	# Solo era Llama y Gemma que seguían en exceso el prompt dado. 
	if llm: # Limpiamos respuesta de un LLM
		for i in range(len(instance)):
			instance[i] = re.sub('(:::)+([ A-z.]+)', '', instance[i])
			instance[i] = re.sub('(  )+', '', instance[i])

	pred_dict = get_nice_dict(r'[A-z]+\(([A-z]+(,? [A-z]+)*)\)', instance, False)
	const_dict = get_nice_dict(r'(\([A-z]+(\, [A-z]+){0,}\))', instance, True)
	logop_dict = get_nice_dict(r'[∀∧→⊕¬∨∃]+', instance, False)

	# len(instance) = Cantidad de premisas
	# pred_dict = Diccionario con cantidad de predicados. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE PREDICADOS
	# constant_dict =  Diccionario con cantidad de constantes. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE CONSTANTES
	# logop_dict =  Diccionario con cantidad de operadores y cuantificadores. DE AQUÍ SE OBTIENE APARICIONES TOTALES Y CANTIDAD DE CUANTIFICADORES
	return pred_dict, const_dict, logop_dict, len(instance)


def total_apps(dict1, preds):
    """
        Obtiene:
            1) La cantidad total de apariciones de un valor (Const/Pred/LogOps)
            2) La cantidad de constantes/predicados/logops distintos.

        dict1 = dict ; El diccionario obtenido previamente.
        preds = bool ; Valor booleano que permite realizar un filtro sobre los predicados.
    """
    if preds:
        lista_chida_aux = [re.search(r'([A-z]+\()', _).group() for _ in dict1.keys()]
        lista_chida = list(set(lista_chida_aux))

        # Guardamos la cantidad de apariciones en el diccionario previo.
        aux_dict = {"{}".format(_):0 for _ in lista_chida}
        for key in dict1.keys():
            for _ in lista_chida:
                if _ in key:
                    aux_dict["{}".format(_)] += dict1[key]
        
        if preds:
            aux_dict = {"{}".format(_[:-1]): aux_dict[_] for _ in aux_dict}
    else:
        aux_dict = dict1
    
    total_value = 0
    for _ in aux_dict:
        total_value += aux_dict[_]

    #total_value = Apariciones totales
    #len(aux_dict) = Cantidad de [VALUE] distinto.
    return total_value, len(aux_dict)


def logic_sim_plus_individual(llm_ans, folio_ans):
    """
        Extrae los valores distintos y valores totales de cada instancia. 
    """
    pred_llm, const_llm, logops_llm, len_llm = extract_info(llm_ans, False)
    pred_folio, const_folio, logops_folio, len_folio = extract_info(folio_ans, True)

    #llm
    dif_preds_llm, pred_count_llm = total_apps(pred_llm, True)
    dif_const_llm, const_count_llm = total_apps(const_llm, False)
    dif_logops_llm, logops_count_llm = total_apps(logops_llm, False)

    #folio
    dif_preds_folio, pred_count_folio = total_apps(pred_folio, True)
    dif_const_folio, const_count_folio = total_apps(const_folio, False)
    dif_logops_folio, logops_count_folio = total_apps(logops_folio, False)

    #Absolute Values
    dif_preds = abs(dif_preds_llm - dif_preds_folio)
    tot_aps_preds = abs(pred_count_llm - pred_count_folio)
    
    dif_const = abs(dif_const_llm - dif_const_folio)
    tot_aps_const = abs(const_count_llm - const_count_folio)

    dif_logops = abs(dif_logops_llm - dif_logops_folio)
    tot_aps_logops = abs(logops_count_llm - logops_count_folio)

    dif_premises = abs(len_llm - len_folio)

    logicsim_plus = dif_preds + tot_aps_preds + dif_const + tot_aps_const + dif_logops + tot_aps_logops + dif_premises
    return logicsim_plus

In [3]:
a = tran_ds['gpt Translations'][0]
b = tran_ds['qwen Translations'][0]
c = tran_ds['premises-FOL'][0]

print("FIRST ORDER LOGIC")
print("=================")
print(a)
print("========")
print(b)
print("========")
print(c)

FIRST ORDER LOGIC
∀x (ProSoccerPlayer(x) → ¬PlayBasketball(x))
∀x (ProSoccerDefender(x) → ProSoccerPlayer(x))
∀x (ProCenterBack(x) → ProSoccerDefender(x))
∃x (Athlete(x) ∧ ProCenterBack(x))



∀x (ProfessionalSoccerPlayer(x) → ¬ProfessionalBasketballPlayer(x))  
∀x (ProfessionalSoccerDefender(x) → ProfessionalSoccerPlayer(x))  
∀x (ProfessionalCenterback(x) → ProfessionalSoccerDefender(x))  
∃x (Athlete(x) ∧ ProfessionalCenterback(x))  
ProfessionalBasketballPlayer(StephenCurry)
∀x ((Professional(x) ∧ SoccerPlayer(x)) → ¬Play(x, professionalBasketball))
∀x ((Professional(x) ∧ Defender(x)) → (Professional(x) ∧ SoccerPlayer(x)))
∀x ((Professional(x) ∧ CenterBack(x)) → (Professional(x) ∧ Defender(x)))
∃x (Athlete(x) ∧ Professional(x) ∧ CenterBack(x))
Play(stephenCurry, professionalBasketball)


In [ ]:
print("GPT vs Qwen: {}".format(logic_sim_plus_individual(a, b)))
print("GPT vs FOLIO: {}".format(logic_sim_plus_individual(a, c)))
print("FOLIO vs Qwen: {}".format(logic_sim_plus_individual(c, b)))
# HMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM

GPT vs Qwen: 5
GPT vs FOLIO: 19
FOLIO vs Qwen: 18
